This notebook  is for those students who want to see the execution of yolo face detection on colab. 

the code provided below is just to use webcam in your laptop if it there. If your laptop does not have a webcam please don't execute this cell.

In [ ]:
#@title Code for accessing webcam
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

Since google colab has both tensorflow 1 and tensorflow 2, we need to ensure that for our purpose tensorflow 1 is used. We can also check the version using:


```
print(tf.__version__)
```



In [ ]:
%tensorflow_version 1.x

In the next we will mount google drive to save our time. Upload the model folder to your drive and after that get into the folder using cd command


```
cd <folderpath>
```





In [ ]:
#@title Mounting drive
from google.colab import drive
drive.mount("/content/gdrive")

In the following code we take image from webcam (click on capture button over the image stream) and pass it into our detector. People who don't have webcam, just comment the first two statements of the while loop and change the string in imread() to your image address/name.


In [ ]:
from IPython.display import Image
from google.colab.patches import cv2_imshow# google colab does not support cv2.imshow() statement , we use a patch provided in its place
from yolo import YOLO #import the model created in yolo.py file
from PIL import Image
import cv2
import numpy as np
yolo=YOLO()
k='y'
while k=='y':
  filename = take_photo()
  print('Saved to {}'.format(filename))
  img=cv2.imread("photo.jpg")#the previous statement saves the picture as photo.jpg
  cv2img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # The storage order of the color hex codes in cv2 and PIL is different
  pilimg = Image.fromarray(cv2img)
  r_image = yolo.detect_image(pilimg)
  cv2charimg = cv2.cvtColor(np.array(r_image), cv2.COLOR_RGB2BGR)
  cv2_imshow(cv2charimg)
  print("Do you want more detections? Press y for more or anyother key to stop execution.")
  k=input()
yolo.close_session()